<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pushover Agent
[code](https://replit.com/@matt/OpenAI-Agents-SDK#main.py)
[video](https://www.youtube.com/watch?v=Ta5J_2KFBGM)
[replit](https://docs.replit.com/getting-started/intro-replit)
[trace](https://platform.openai.com/traces)

In [1]:
# Install

!pip install agents
!pip install openai-agents

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 21.3 MB/s eta 0:00:00
  Created wheel for agents: filename=agents-1.4.0-py3-none-any.whl size=62714 sha256=81efc26b33f97933f946d446061e02c8902b4f39bb9302586bf0aa1d3f2abba1
  Stored in directory: /root/.cache/pip/wheels/e5/82/e5/2790dbbc1ad6037f1001bc436ea963e0877fff918dddc74fe2
Successfully built agents
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.0 MB/s eta 0:00:00


In [2]:
# Imports

import os
from datetime import datetime
from agents import Agent, Runner, function_tool, trace
import gradio as gr
from google.colab import userdata
from datetime import datetime
import httpx
from bs4 import BeautifulSoup
from typing_extensions import TypedDict, Any

In [3]:
# Fetch tokens for openai and pushover

openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and starts with {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

pushover_user = userdata.get("PUSHOVER_USER")
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

pushover_token = userdata.get("PUSHOVER_TOKEN")
if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

pushover_url = "https://api.pushover.net/1/messages.json"

OpenAI API Key exists and starts with sk-proj-
Pushover user found and starts with u
Pushover token found and starts with a


In [4]:
# The usual starting point

os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
# Make an agent with name, instructions, model

async def main_test():
  agent = Agent(name="Jokester2",
            instructions="You are a joke teller",
            model="gpt-4o-mini")

  with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

await main_test()

Why did the Autonomous AI Agent break up with its computer partner?

Because it needed more space... and a better algorithm for love!


In [6]:
# Here is the modified code to run in Colab

class Location(TypedDict):
    lat: float
    long: float

@function_tool
async def get_time() -> str:
    """Fetch the current time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@function_tool
async def fetch_url(url: str) -> str:
    """Fetch content from a URL"""
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        return response.text

@function_tool
async def fetch_weather(location: Location) -> str:
    """Fetch the weather for a given location."""
    return "sunny"

async def main():
    """Run the agent"""
    agent = Agent(
        name="Assistant",
        model="gpt-4o-mini",
        instructions="You are a helpful assistant",
        tools=[get_time, fetch_url]
    )

    result = await Runner.run(
        agent, """
            Please get the content of https://sullivanexcavatinginc.com/ and summarize them.
            Return the summary as well as the time you got the content.
            """
    )
    print(result.final_output)

await main()

I was unable to extract meaningful content from the website https://sullivanexcavatinginc.com/ as it appears to be built with dynamic content, and the page returned only its basic HTML structure without providing any visible information.

The current time when I attempted to access the content is **December 29, 2025, 13:28:47**.


# Task
Develop an agent capable of fetching content from a URL, assessing its readability, and sending a Pushover notification if the content is deemed unreadable (e.g., less than 200 characters of text after HTML parsing).

## send_pushover_notification

### Subtask:
Define an asynchronous helper function to send messages to Pushover using the global `pushover_user`, `pushover_token`, and `pushover_url`.


**Reasoning**:
Define an asynchronous helper function `send_pushover_notification` to send messages to Pushover, utilizing `httpx.AsyncClient` for the POST request and global Pushover credentials.



In [7]:
async def send_pushover_notification(message: str):
    """Sends a notification to Pushover."""
    async with httpx.AsyncClient() as client:
        try:
            response = await client.post(
                pushover_url,
                data={
                    "token": pushover_token,
                    "user": pushover_user,
                    "message": message,
                },
            )
            response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
            print(f"Pushover notification sent successfully: {message}")
        except httpx.HTTPStatusError as e:
            print(f"Error sending Pushover notification: HTTPStatusError - {e.response.status_code} {e.response.text}")
        except httpx.RequestError as e:
            print(f"Error sending Pushover notification: RequestError - {e}")

print("Defined send_pushover_notification function.")

Defined send_pushover_notification function.


## check_and_notify_unreadable_content

### Subtask:
Create a new `function_tool` that takes HTML content and a URL as input. It will parse the HTML using `BeautifulSoup` to extract readable text. If the extracted text length falls below a defined threshold (e.g., 200 characters), it will consider the content unreadable and call `send_pushover_notification`. It will return a string indicating whether the content was readable or not, along with a message.


**Reasoning**:
The subtask requires defining a new `function_tool` to check HTML content readability and send a Pushover notification if it's unreadable. This code block implements the `check_and_notify_unreadable_content` function with the specified logic.



In [8]:
@function_tool
async def check_and_notify_unreadable_content(html_content: str, url: str) -> str:
    """Checks if HTML content is readable (based on text length) and sends a Pushover notification if not."""
    soup = BeautifulSoup(html_content, 'html.parser')
    extracted_text = soup.get_text()

    # Clean the extracted text
    cleaned_text = ' '.join(extracted_text.split()).strip()

    readability_threshold = 200  # Define readability threshold

    if len(cleaned_text) < readability_threshold:
        notification_message = f"Content from {url} is unreadable (too short: {len(cleaned_text)} characters)."
        await send_pushover_notification(notification_message)
        return f"Content from {url} is unreadable (too short: {len(cleaned_text)} characters). Pushover notification sent."
    else:
        return f"Content from {url} is readable (length: {len(cleaned_text)} characters)."

print("Defined check_and_notify_unreadable_content function.")

Defined check_and_notify_unreadable_content function.


**Reasoning**:
The subtask has been completed by defining the `check_and_notify_unreadable_content` function. Now I will update the main agent to use the newly defined `check_and_notify_unreadable_content` tool along with `fetch_url` to achieve the overall goal of checking URL readability and sending notifications.



In [9]:
async def main_with_readability_check():
    """Run the agent to fetch URL content, check readability, and notify if unreadable."""
    agent = Agent(
        name="ContentChecker",
        model="gpt-4o-mini",
        instructions="You are an agent that fetches content from a URL, checks its readability, and sends a notification if the content is unreadable (less than 200 characters of text after HTML parsing).",
        tools=[get_time, fetch_url, check_and_notify_unreadable_content]
    )

    print("\n--- Testing with an unreadable URL ---")
    result_unreadable = await Runner.run(
        agent,
        "Please fetch the content of https://sullivanexcavatinginc.com/, then check its readability and notify if unreadable."
    )
    print(result_unreadable.final_output)

    print("\n--- Testing with a readable URL ---")
    result_readable = await Runner.run(
        agent,
        "Please fetch the content of https://www.google.com/, then check its readability and notify if unreadable."
    )
    print(result_readable.final_output)

await main_with_readability_check()


--- Testing with an unreadable URL ---
Pushover notification sent successfully: Content from https://sullivanexcavatinginc.com/ is unreadable (too short: 23 characters).
The content from [Sullivan Excavating Inc](https://sullivanexcavatinginc.com/) is unreadable, containing only 23 characters. A notification has been sent regarding this issue.

--- Testing with a readable URL ---
Pushover notification sent successfully: Content from https://www.google.com/ is unreadable (too short: 6 characters).
The content from [Google](https://www.google.com/) was found to be unreadable, containing only 6 characters. A Pushover notification has been sent regarding this issue.


## Summary:

### Data Analysis Key Findings
*   An asynchronous function `send_pushover_notification` was successfully defined, capable of sending messages to Pushover using `httpx.AsyncClient`, with built-in error handling for HTTP and request-related issues.
*   A `function_tool` named `check_and_notify_unreadable_content` was created. This tool utilizes `BeautifulSoup` to parse HTML, extract, and clean text, and then checks if the cleaned text length falls below a 200-character readability threshold. If deemed unreadable, it triggers the `send_pushover_notification` function.
*   Testing with `https://sullivanexcavatinginc.com/` confirmed its content as unreadable, with only 23 characters of extracted text, and a Pushover notification was successfully sent.
*   Testing with `https://www.google.com/` also resulted in the content being identified as unreadable, with only 6 characters of extracted text, leading to a Pushover notification. This demonstrates that the readability assessment strictly adheres to the raw text character count after HTML parsing.

### Insights or Next Steps
*   The current readability assessment, based solely on character count, may produce false positives for pages that are visually rich or interactive with minimal raw text content (e.g., `https://www.google.com/`).
*   To improve accuracy, consider integrating more advanced readability metrics or filtering mechanisms that focus on visible, meaningful text content, rather than just the total extracted character count from raw HTML.
